In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
import sys, os
sys.path.append("/workspace/src")
import argparse
import gymnasium as gym
import pandas as pd
from src.training.setup import setup_env

In [7]:
args_dict = {"env": "CartPole-v1"}
args = argparse.Namespace(**args_dict)

In [8]:
config = {}
env = setup_env(config=config, args=args)

In [18]:
state_column = ["cart_position", "cart_velocity", "pole_angle", "pole_angular_velocity"]

df = pd.DataFrame([], columns=["state", "action", "next_state", "reward", "done"])

In [11]:
observation, info = env.reset()

for _ in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    next_observation, reward, terminated, truncated, info = env.step(action)
    
    if terminated or truncated:
        observation, info = env.reset()
    
    {"state": observation, "action": action, "next_state": next_observation, "reward": reward }

env.close()

In [12]:
observation

array([ 0.06244553,  0.82431614, -0.10735495, -1.1336427 ], dtype=float32)